In [1]:
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams.update({'font.size': 18})
matplotlib.rcParams['lines.linewidth'] = 0.5
matplotlib.rcParams['lines.markeredgewidth'] = 0.5
matplotlib.rcParams['lines.markersize'] = 8 

In [2]:
import numpy as np
from numba import njit, prange, set_num_threads
from itertools import product

set_num_threads(15)

In [3]:
from ns2d import (
    EulerIntegrator,
    FiniteDifferenceDiscretizer,
    FiniteDifferenceUpwindDiscretizer,
    FiniteVolumeDiscretizer,
    GaussSeidelSolver,
    JacobiSolver,
    PredictorCorrectorIntegrator,
    RK4Integrator
)

In [4]:
nx, ny = 41, 41
dx, dy = 2.0 / (nx - 1), 2.0 / (ny - 1)
dt = 0.001
nu = 1e-2
num_steps = 500

x = np.linspace(0, 2, nx)
y = np.linspace(0, 2, ny)
X, Y = np.meshgrid(x, y)

In [5]:
rhs_discretizers = [
    FiniteDifferenceDiscretizer(),
    FiniteDifferenceUpwindDiscretizer(),
    FiniteVolumeDiscretizer()
]
predictors = [
    EulerIntegrator(),
    PredictorCorrectorIntegrator,
    RK4Integrator
]
benchmarks = {
    "Taylor-Green Vortex": 1,
    "Lid-Driven Cavity": 2
}
all_combinations = list(product(rhs_discretizers, predictors))

In [ ]:
solver = JacobiSolver(
    nx = nx, ny=ny, dx=dx, dy=dy, dt=dt, nu=nu,
    integrator=EulerIntegrator(),
    discrete_navier_stokes=FiniteDifferenceDiscretizer(),
    bc_case=benchmarks["Taylor-Green Vortex"],
    fixed_dt=False
)
errors = solver.integrate(
    num_steps=10000,
    end_time=None,
    benchmark="Taylor-Green Vortex"
)
vorticity = solver.compute_vorticity()

In [ ]:
time = [item[0] for item in errors]
error_values = [item[1] for item in errors]
plt.plot(time, error_values, label='Error in kinetic energy')
plt.xlabel('Time')
plt.ylabel('Error')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
cf = plt.contourf(X, Y, vorticity)
cb = plt.colorbar(cf)
cb.set_label('Vorticity (s⁻¹)')
plt.xlabel('x')
plt.ylabel('y')
plt.tight_layout()
plt.show()

cf = plt.contourf(X, Y, solver.p)
cb = plt.colorbar(cf)
cb.set_label('Pressure (Pa)')
plt.xlabel('x')
plt.ylabel('y')
plt.tight_layout()
plt.show()

# Results for Jacobi Solver

In [ ]:
# for discretizer, predictor in combinations:
#     rhs_name = discretizer.__class__.__name__
#     lhs_name = predictor.__class__.__name__
    
#     print(f"\nTesting combination:")
#     print(f" - Discretizer: {rhs_name}")
#     print(f" - Predictor: {lhs_name}")

#     # Run for both benchmarks
#     try:
#         # Run actual simulation
#         # ...
        
#         # Print or store the result
#         # results[(rhs_name, lhs_name)] = result
        
#     except Exception as e:
#         print(f"Error occurred: {e}")

In [6]:
# plt.figure(figsize=(11, 7))
# plt.contourf(X, Y, solver.p, alpha=0.5)
# plt.colorbar(label='Pressure')
# plt.contour(X, Y, solver.p, colors='black', linestyles='solid')
# plt.quiver(X[::2, ::2], Y[::2, ::2], solver.u[::2, ::2], solver.v[::2, ::2], color='r')
# plt.xlabel('X')
# plt.ylabel('Y')
# plt.tight_layout()
# plt.show()

# Results for Gauss-Seidel Solver